## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
import bert_tokenization as tokenization
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH_TF = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH_TF = BERT_PATH_TF + '/assets/vocab.txt'
BERT_PATH = '/kaggle/input/bert-base-uncased-huggingface/bert-base-uncased/bert-base-uncased-tf_model.h5'
VOCAB_PATH = '/kaggle/input/bert-base-uncased-huggingface/bert-base-uncased/bert-base-uncased-vocab.txt'

model_path_list_tf = glob.glob('/kaggle/input/130-googleq-a-train-3fold-bert-base-unc-quest-ans3/' + '*.h5')
model_path_list_tf += glob.glob('/kaggle/input/bert-131-last/bert-131-last/' + '*.h5')
model_path_list_tf += glob.glob('/kaggle/input/bert-131-best/' + '*.h5')
model_path_list_tf.sort()

model_path_list_tf2 = glob.glob('/kaggle/input/bert-138-last/' + '*.h5')
model_path_list_tf2.sort()

model_path_list = glob.glob('/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/' + '*1.h5')
model_path_list += glob.glob('/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/' + '*2.h5')
model_path_list += glob.glob('/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/' + '*3.h5')
model_path_list.sort()
n_models  = len(model_path_list_tf) + len(model_path_list_tf2) + len(model_path_list)

print('Models to predict (tf):')
print(*model_path_list_tf, sep = "\n")
print('Models to predict (tf2):')
print(*model_path_list_tf2, sep = "\n")
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict (tf):
/kaggle/input/130-googleq-a-train-3fold-bert-base-unc-quest-ans3/model_fold_1.h5
/kaggle/input/130-googleq-a-train-3fold-bert-base-unc-quest-ans3/model_fold_2.h5
/kaggle/input/130-googleq-a-train-3fold-bert-base-unc-quest-ans3/model_fold_3.h5
/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_1.h5
/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_2.h5
/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_3.h5
/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_4.h5
/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_5.h5
/kaggle/input/bert-131-last/bert-131-last/131-Bert_base_unc Quest_Ans3_model_fold_1_last_epoch.h5
/kaggle/input/bert-131-last/bert-131-last/131-Bert_base_unc Quest_Ans3_model_fold_2_last_epoch.h5
/kaggle/input/bert-131-last/bert-131-last/131-Bert_base_unc Quest_Ans3_model_fold_4_last_epoch.h5
/kaggle/input/bert-131-last/bert-131-last/131-Bert_base_unc Quest_Ans3_mod

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

In [4]:
### BERT auxiliar function
def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length, ignore_first_sep=True):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if ignore_first_sep:
                ignore_first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input_2_tail(ft_input1, ft_input2, tokenizer, max_sequence_length, 
                  max_len_ft1=254, max_len_ft2=255):

    ft1 = tokenizer.tokenize(ft_input1)
    ft2 = tokenizer.tokenize(ft_input2)
    
    ft1_len = len(ft1)
    ft2_len = len(ft2)

    if (ft1_len + ft2_len + 3) > max_sequence_length:
      
        if max_len_ft2 > ft2_len:
            ft2_new_len = ft2_len 
            ft1_new_len = max_len_ft1 + (max_len_ft2 - ft2_len)
        elif max_len_ft1 > ft1_len:
            ft2_new_len = max_len_ft2 + (max_len_ft1 - ft1_len)
            ft1_new_len = ft1_len
        else:
            ft2_new_len = max_len_ft2
            ft1_new_len = max_len_ft1
                        
        if ft1_new_len + ft2_new_len + 3 > max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (ft1_new_len + ft2_new_len + 3)))
        
        ft1 = ft1[-ft1_new_len:]
        ft2 = ft2[-ft2_new_len:]
        
    return ft1, ft2

def _convert_to_bert_inputs_2(feature1, feature2, tokenizer, max_sequence_length, ignore_first_sep=True):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + feature1 + ["[SEP]"] + feature2 + ["[SEP]"]

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length, ignore_first_sep)

    return [input_ids, input_masks, input_segments]

def compute_input_arays_2_tail(df, columns, tokenizer, max_sequence_length, 
                          max_len_ft1, max_len_ft2, ignore_first_sep=True):
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in df[columns].iterrows():
        ft1, ft2 = instance[columns[0]], instance[columns[1]]

        ft1, ft2 = _trim_input_2_tail(ft1, ft2, tokenizer, max_sequence_length, 
                                      max_len_ft1, max_len_ft2)

        ids, masks, segments = _convert_to_bert_inputs_2(ft1, ft2, tokenizer, max_sequence_length, 
                                                         ignore_first_sep)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
        
    return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

## Pre-process data

In [5]:
text_features = ['question_title', 'question_body', 'answer']
text_features_question = ['question_title', 'question_body']
text_features_answer = ['question_title', 'answer']

# Model parameters

In [6]:
N_CLASS = len(target_cols)
N_CLASS_QUESTION = len(question_target_cols)
N_CLASS_ANSWER = len(answer_target_cols)
MAX_SEQUENCE_LENGTH = 512
MAX_LENGTH_TITLE = 30
MAX_LENGTH_QUESTION = 479
MAX_LENGTH_ANSWER = 479

## Test set

In [7]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH_TF, do_lower_case=True)

# Test features HEAD
X_test = compute_input_arays_2(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                               MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)\
        + compute_input_arays_2(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Test features TAIL
X_test_tail = compute_input_arays_2_tail(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                                         MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)\
              + compute_input_arays_2_tail(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                           MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Model

In [8]:
bert_config = BertConfig()
bert_config.output_hidden_states=False

def model_fn():
    input_word_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_question')
    input_masks_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_question')
    segment_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_question')

    input_word_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_answer')
    input_masks_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_answer')
    segment_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_answer')

    bert_model = TFBertModel.from_pretrained(BERT_PATH, config=bert_config)
    sequence_output_question, pooled_output_question = bert_model([input_word_ids_question, input_masks_question, segment_ids_question])
    sequence_output_answer, pooled_output_answer = bert_model([input_word_ids_answer, input_masks_answer, segment_ids_answer])
    
    x_question = GlobalAveragePooling1D()(sequence_output_question)
    x_question = Dropout(0.2)(x_question)
    output_question = Dense(N_CLASS_QUESTION, kernel_initializer='glorot_uniform', activation="sigmoid", name="outputquestion")(x_question)
    
    x_answer = GlobalAveragePooling1D()(sequence_output_answer)
    x_answer = Dropout(0.2)(x_answer)
    output_answer = Dense(N_CLASS_ANSWER, kernel_initializer='glorot_uniform', activation="sigmoid", name="output_answer")(x_answer)
    
    output = Concatenate()([output_question, output_answer])

    model = Model(inputs=[input_word_ids_question, input_masks_question, segment_ids_question, 
                          input_word_ids_answer, input_masks_answer, segment_ids_answer], outputs=output)

    return model

def model_fn_tf():
    input_word_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_question')
    input_masks_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_question')
    segment_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_question')

    input_word_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_answer')
    input_masks_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_answer')
    segment_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_answer')

    bert_layer = KerasLayer(BERT_PATH_TF, trainable=True)
    pooled_output_question, sequence_output_question = bert_layer([input_word_ids_question, input_masks_question, segment_ids_question])
    pooled_output_answer, sequence_output_answer = bert_layer([input_word_ids_answer, input_masks_answer, segment_ids_answer])

    x_question = GlobalAveragePooling1D()(sequence_output_question)
    x_question = Dropout(0.2)(x_question)
    output_question = Dense(N_CLASS_QUESTION, kernel_initializer='glorot_uniform', activation="sigmoid", name="outputquestion")(x_question)
    
    x_answer = GlobalAveragePooling1D()(sequence_output_answer)
    x_answer = Dropout(0.2)(x_answer)
    output_answer = Dense(N_CLASS_ANSWER, kernel_initializer='glorot_uniform', activation="sigmoid", name="output_answer")(x_answer)
    
    output = Concatenate()([output_question, output_answer])

    model = Model(inputs=[input_word_ids_question, input_masks_question, segment_ids_question, 
                          input_word_ids_answer, input_masks_answer, segment_ids_answer], outputs=output)

    return model


def model_fn_tf2():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH_TF, trainable=False)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [9]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += ((model.predict(X_test) + model.predict(X_test_tail)) / 2) / n_models
    
for model_path in model_path_list_tf:
    model = model_fn_tf()
    model.load_weights(model_path)
    Y_test += ((model.predict(X_test) + model.predict(X_test_tail)) / 2) / n_models
    
for model_path in model_path_list_tf2:
    model = model_fn_tf2()
    model.load_weights(model_path)
    Y_test += ((model.predict(X_test) + model.predict(X_test_tail)) / 2) / n_models

In [10]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test


# n=test['url'].apply(lambda x:(('ell.stackexchange.com' in x) or ('english.stackexchange.com' in x))).tolist()
# spelling=[]
# for x in n:
#     if x:
#         spelling.append(0.5)
#     else:
#         spelling.append(0.)
        
# submission['question_type_spelling']=spelling


submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.953507,0.695183,0.162824,0.616901,0.641201,0.603509,0.686106,0.638112,0.516880,...,0.911559,0.934077,0.626078,0.973088,0.976435,0.846252,0.043377,0.038125,0.822683,0.916932
1,46,0.861798,0.500108,0.008709,0.779624,0.774629,0.934871,0.536041,0.457276,0.087832,...,0.641709,0.940234,0.663171,0.970750,0.979680,0.871423,0.879715,0.128230,0.145817,0.877207
2,70,0.895262,0.690728,0.028490,0.789512,0.878969,0.920898,0.586120,0.490428,0.131377,...,0.859073,0.948565,0.632484,0.969904,0.970908,0.870428,0.050321,0.043115,0.864238,0.907203
3,132,0.839173,0.419477,0.009314,0.708116,0.760134,0.911453,0.508830,0.399409,0.100994,...,0.669735,0.953588,0.703736,0.971149,0.980632,0.906637,0.772888,0.159859,0.601196,0.887457
4,200,0.932315,0.478202,0.037392,0.876551,0.791687,0.909015,0.637064,0.591562,0.085856,...,0.659374,0.922701,0.664268,0.960629,0.965830,0.840505,0.205367,0.129715,0.631912,0.886449


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.891478,0.602740,0.032761,0.751046,0.811013,0.861495,0.576797,0.487319,0.197883,...,0.785990,0.934192,0.665800,0.964078,0.972931,0.869153,0.497816,0.134083,0.510610,0.903609
std,2812.670060,0.047502,0.145066,0.050820,0.087136,0.089694,0.112949,0.052694,0.087300,0.168569,...,0.096134,0.023018,0.037592,0.011884,0.010704,0.033756,0.301487,0.057888,0.252673,0.022140
min,39.000000,0.742782,0.346997,0.004337,0.382506,0.459894,0.231130,0.485198,0.358900,0.014039,...,0.574367,0.810831,0.547739,0.908454,0.901203,0.737421,0.008252,0.012815,0.061005,0.832567
25%,2572.000000,0.855306,0.470893,0.008803,0.704890,0.770341,0.846749,0.533908,0.420819,0.070075,...,0.700773,0.921485,0.640350,0.958518,0.968029,0.851144,0.191527,0.095463,0.290739,0.889493
50%,5093.000000,0.895825,0.588653,0.012706,0.752669,0.820246,0.904952,0.566549,0.457755,0.135823,...,0.781937,0.939276,0.664306,0.966194,0.975025,0.872678,0.581506,0.137654,0.504803,0.905576
75%,7482.000000,0.929726,0.732399,0.028791,0.800828,0.871122,0.927567,0.614897,0.542505,0.289610,...,0.878785,0.950283,0.689043,0.972222,0.980509,0.892696,0.771623,0.174161,0.725493,0.919333
max,9640.000000,0.976061,0.899250,0.386679,0.963952,0.974981,0.970100,0.716034,0.734911,0.796849,...,0.959722,0.977856,0.786994,0.983402,0.990115,0.939564,0.933787,0.305896,0.976253,0.953157
